In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# FOLDERNAME = 'CS231n_project/'
# assert FOLDERNAME is not None, "[!] Enter the foldername."

# import sys
# sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# %cd /content/drive/My\ Drive/$FOLDERNAME

# %load_ext autoreload
# %autoreload 2

In [2]:
# !pip install torch==1.7 torchvision==0.8

In [3]:
# %cd approx/src/pytorch/cpp
# !python setup.py install
# %cd ../../../..

In [6]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from conv_norm import PreConv

import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F

import numpy as np
from timeit import default_timer as timer
from utils import ImportanceSampler

USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [7]:
from utils import get_accuracy, load_dataset
from models import get_model
check_accuracy = lambda loader, model: get_accuracy(loader, model, device, dtype)

In [8]:
def train_model(model_name, dataset_name, model_params={}, hyperparams={}):

  learning_rate = hyperparams.get('lr', 1e-3)
  num_epochs = hyperparams.get('num_epochs', 10)
  weight_decay = hyperparams.get('weight_decay', 0)
  train_ratio = hyperparams.get('train_ratio', 0.8)
  batch_size = hyperparams.get('batch_size', 64)
  seed = hyperparams.get('seed', 0)
  imp_sampling = model_params.get('importance_sampling', False)
  gamma = model_params.get('gamma', 0.9)

  torch.manual_seed(seed)
  np.random.seed(seed)

  loader_train, loader_val, loader_test, num_train, num_channels = load_dataset(dataset_name, train_ratio, batch_size)
  model = get_model(model_name, model_params, learning_rate, loader_train, num_channels, device)

  print("Model architecture:")
  print(model)

  print(f'INFO: Training {model_name} on {dataset_name} with lr {learning_rate}, num_epochs={num_epochs}, weight_decay={weight_decay}')

  optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0)

  epoch_vals = []
  
  weight = torch.tensor([1.0]*num_train)

  t_acc, t_loss = check_accuracy(loader_train, model)
  val_acc, val_loss = check_accuracy(loader_val, model)
  
  start = timer()
  c_time = timer()-start

  print(f'Plot: Train, {0}, {t_loss:.3f}, {t_acc:.2f}, {c_time:.1f}')
  print(f'Plot: Val, {0}, {val_loss:.3f}, {val_acc:.2f}, {c_time:.1f}')

  for e in range(num_epochs):
    model.train()
    doUniform = (e == 0) or (imp_sampling == False)
    loader_train_sampled = loader_train
    if not doUniform:
      train_sampler = ImportanceSampler(num_train, weight, batch_size)
      loader_train_sampled, _, _, _, _ = load_dataset(dataset_name, train_ratio, batch_size, train_sampler)
    
    for t, tpl in enumerate(loader_train_sampled):
        torch.cuda.empty_cache()
        model.train()  # put model to training mode
        x = tpl[0].to(device=device, dtype=dtype)  # move to device, e.g. GPU
        y = tpl[1].to(device=device, dtype=torch.long)

        scores = model(x)
        loss = F.cross_entropy(scores, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if not doUniform:
          idx = tpl[2]
          weight[idx] = gamma * weight[idx] + (1 - gamma) * float(loss)

    t_acc, t_loss = check_accuracy(loader_train, model)
    model.eval()
    val_acc, val_loss = check_accuracy(loader_val, model)
    c_time = timer()-start

    print(f'Plot: Train, {e+1}, {t_loss:.3f}, {t_acc:.2f}, {c_time:.1f}')
    print(f'Plot: Val, {e+1}, {val_loss:.3f}, {val_acc:.2f}, {c_time:.1f}')

  test_acc, test_loss = check_accuracy(loader_test, model)
  print(f'Plot: Test, {val_loss:.3f}, {val_acc:.2f}, {c_time:.1f}')

  return model

In [9]:
# learning_rate = hyperparams.get('lr', 1e-3)
# num_epochs = hyperparams.get('num_epochs', 10)
# weight_decay = hyperparams.get('weight_decay', 0)
# train_ratio = hyperparams.get('train_ratio', 0.8)
# batch_size = hyperparams.get('batch_size', 64)
# imp_sampling = model_params.get('importance_sampling', False)
# gamma = model_params.get('gamma', 0.9)

# gradinit_params = {
#     "gradinit_iters": 50,
# }
model_params = {
    # "gradinit": gradinit_params,
    "convnorm" : {"mode_conv": [('first_frac', 0.25)], "mode_bn": [('first_frac', 0.25)]},
    # "approx_mult" : 0.2,
    # "importance_sampling" : True,
    # "gamma" : 0.9
}
hyperparams = {
    "lr" : 3e-3,
    "num_epochs" : 25,
    "weight_decay" : 0,
    "train_ratio" : 0.8,
    "batch_size" : 1024,
}

# train_model('Resnet18', 'CIFAR100', model_params, hyperparams)
# model_params["convnorm"] = True
# train_model('Resnet18', 'CIFAR100', model_params, hyperparams)

def test_setup():
    modeList = [[('first_num', 1)], [('first_frac', 0.25)], [('first_frac', 0.5)], [('first_frac', 0.75)], [('first_frac', 1.0)]]
    for mode in modeList:
        model_params['convnorm']["mode_conv"] = mode
        model_params['convnorm']["mode_bn"] = mode
        train_model('Resnet18', 'CIFAR100', model_params, hyperparams)

test_setup()

Files already downloaded and verified
CIFAR100 Train dataset raw mean: 0.4783550798892975, raw std dev: 0.2678655982017517
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
INFO: Size of dataset: Training 40000, Validation 10000, Test 10000
bn_pres is True
Originally num layers were 20
Replace layers list is: [0]
Number of og layers are 20, number of new layers are 1
bn_pres is True
Originally num layers were 20
Replace layers list is: [0]
Number of og layers are 19, number of new layers are 1
Model architecture:
ResNet(
  (conv1): PreConv(
    3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3)
    (bpconv): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=64)
  )
  (bn1): Identity()
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), s

/home/ubuntu/CS231n-Faster-Classifier/src/conv_norm.py:112: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/aten/src/ATen/native/SpectralOps.cpp:590.)
  f_input = torch.rfft(cout, 2, normalized=False, onesided=True)
/home/ubuntu/CS231n-Faster-Classifier/src/conv_norm.py:123: UserWarning: The function torch.irfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.ifft or torch.fft.irfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/aten/src/ATen/native/SpectralOps.cpp:602.)
  output = torch.irfft(compl_mul_2D(f_input, V), 2, normalized=False, signal_sizes=(final_size_x,final_size_y))


Plot: Train, 0, 6.908, 0.00, 0.0
Plot: Val, 0, 6.907, 0.00, 0.0
Plot: Train, 1, 3.785, 10.80, 112.2
Plot: Val, 1, 4.088, 9.56, 112.2
Plot: Train, 2, 3.359, 17.44, 225.6
Plot: Val, 2, 3.801, 15.54, 225.6


KeyboardInterrupt: 

In [ ]:
def test_setup2():
    modeList = [[('last_num', 1)], [('last_frac', 0.25)], [('last_frac', 0.5)], [('last_frac', 0.75)]]
    for mode in modeList:
        model_params['convnorm']["mode_conv"] = mode
        model_params['convnorm']["mode_bn"] = mode
        train_model('Resnet18', 'CIFAR100', model_params, hyperparams)

test_setup2()